In [14]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv("task2_allData.csv")
test_public = pd.read_csv("task2_public_test_data.csv")
test = pd.read_csv("task2_private_test_data.csv")

In [6]:
test = pd.merge(test_public, test, how = "outer")

In [7]:
x_train = train.drop(["AnswerValue", "AnswerId"], axis=1)
y_train = train.AnswerValue
x_test = test.drop(["AnswerValue", "AnswerId"], axis=1)
y_test = test.AnswerValue

In [8]:
x_train["SchemeOfWorkId"] = x_train["SchemeOfWorkId"].astype(int)
x_train["PremiumPupil"] = x_train["PremiumPupil"].astype(int)
x_train["Confidence"] = x_train["Confidence"].astype(int)

x_test["SchemeOfWorkId"] = x_test["SchemeOfWorkId"].astype(int)
x_test["PremiumPupil"] = x_test["PremiumPupil"].astype(int)
x_test["Confidence"] = x_test["Confidence"].astype(int)

In [11]:
model = CatBoostClassifier(
    iterations=4,
    # learning_rate=0.8,
    # depth=16,
    cat_features=["UserId", "QuestionId", "QuizId", "GroupId", "SchemeOfWorkId",
                  "Gender", "PremiumPupil", "Confidence", "MathType", "SchoolShift"],
    # max_leaves=None,
    # num_leaves=None,
    # min_data_in_leaf=None,
    # min_child_samples=None,
    # max_bin=700,
    # num_trees=None,
    # max_depth=15,
    # n_estimators=500,
    verbose=2
)

In [12]:
model.fit(x_train, y_train)

y_predict = model.predict(x_test)
print('private ', accuracy_score(y_test, y_predict))

Learning rate set to 0.5
0:	learn: 1.0168628	total: 20.8s	remaining: 1m 2s
2:	learn: 0.9317434	total: 1m 3s	remaining: 21s
3:	learn: 0.9196588	total: 1m 22s	remaining: 0us
private  0.645774109816502


In [15]:
print(classification_report(y_test, y_predict, target_names=["A", "B", "C", "D"]))

              precision    recall  f1-score   support

           A       0.65      0.65      0.65    957034
           B       0.65      0.64      0.65   1037573
           C       0.65      0.65      0.65   1049075
           D       0.63      0.65      0.64    923281

    accuracy                           0.65   3966963
   macro avg       0.65      0.65      0.65   3966963
weighted avg       0.65      0.65      0.65   3966963



In [6]:
import pickle
pickle.dump(model, open("CatBoost.sav", 'wb'))

In [18]:
df = pd.DataFrame(y_predict, columns =['CatBoost'])
df.to_csv("CatBoost2.csv", header=True, index=False)